In [37]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

import torch


if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

model = T5ForConditionalGeneration.from_pretrained('t5-base')

model.load_state_dict(torch.load('t5_09_epoch_10.pt', map_location=device))

<All keys matched successfully>

In [38]:
task_prefix = 'grammar: '
# sentence = "It 's difficult answer at the question "" what are you going to do in the future ? "" if the only one who has to know it is in two minds ."
sentence = "Nowadays in the cities developed there are many possibilities for public transport , like bus , train , subway , tram etc.these options it 's more economical than the car , the problem of the car is that normally the car is occupied for a person , when the car is prepared for to carry four or five people ."

t5_tokenizer = T5Tokenizer.from_pretrained('t5-base')

input_ids = t5_tokenizer(task_prefix + sentence, return_tensors='pt').input_ids.to(device)
outputs = model.generate(input_ids, max_length=512)
corrected_sentence = t5_tokenizer.decode(outputs[0], skip_special_tokens=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [39]:
f = open("../ABCN_dev.src")

orig_sentences = f.readlines()
print(len(orig_sentences))

4384


In [44]:
from tqdm import tqdm

generated_sentences = []
t5_tokenizer = T5Tokenizer.from_pretrained('t5-base')

task_prefix = "rectify the grammatical errors: "

for i in tqdm(range(len(orig_sentences))):
    input_ids = t5_tokenizer(task_prefix + orig_sentences[i], return_tensors='pt').input_ids.to(device)

    outputs = model.generate(input_ids, max_length=100)
    corrected = t5_tokenizer.decode(outputs[0], skip_special_tokens=True)
    generated_sentences.append(corrected)



Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
  3%|▎         | 123/4384 [00:35<20:17,  3.50it/s]


KeyboardInterrupt: 

In [42]:
print(len(generated_sentences))

4384


In [43]:
from tqdm import tqdm 
import spacy 

nlp = spacy.load("en_core_web_sm") 
f = open('t5_prefix_tuning.txt', 'w')

for output_sentence in generated_sentences:
    if output_sentence[0] == '"' and output_sentence[-1] == '"':
        output_sentence = output_sentence[1:-1]
    
    tokens = nlp(output_sentence)
    
    print(list(tokens))

    for token in tokens:
        if '" "' not in str(token):
            f.write(str(token) + " ")
        
    f.write("\n")

f.close()

[What, are, you, going, to, do, in, the, future, ?, ", ,, is, a, difficult, answer, to, the, question, ", what, are, you, going, to, do, in, the, future, ?, ", if, the, only, one, who, has, to, know, it, is, in, two, minds, .]
[When, I, was, younger, ,, I, used, to, say, that, I, wanted, to, be, a, teacher, ,, a, saleswoman, and, even, a, butcher, .., I, do, n't, know, why, .]
[I, would, like, to, study, Psychology, because, one, day, I, would, open, my, own, psychology, office, and, help, people, .]
[It, 's, difficult, because, I, 'll, have, to, study, hard, and, a, lot, ,, but, I, think, that, if, you, like, a, subject, ,, you, 'll, study, it, easier, .]
[Maybe, I, 'll, change, my, mind, ,, maybe, not, .]
[I, think, that, public, transport, will, always, be, in, the, future, .]
[The, rich, people, will, buy, a, car, but, the, poor, people, always, need, to, use, a, bus, or, taxi, .]
[I, consider, that, it, is, more, convenient, to, drive, a, car, because, you, carry, on, more, things